In [ ]:
# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score
from matplotlib import pyplot
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, weights=[1,1], random_state=1)
#print(X[:3],y[:3])
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = KNeighborsClassifier(n_neighbors=3)
model.fit(trainX, trainy)
# predict probabilities
probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# print(probs)
# predict class values
yhat = model.predict(testX)
# calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(testy, probs)
# calculate F1 score
print(thresholds)
f1 = f1_score(testy, yhat)
# calculate precision-recall AUC
auc = auc(recall, precision)
# calculate average precision score
ap = average_precision_score(testy, probs)
print('f1=%.3f auc=%.3f ap=%.3f' % (f1, auc, ap))
# plot no skill
pyplot.plot([0, 1], [0.5, 0.5], linestyle='--')
# plot the precision-recall curve for the model
pyplot.plot(recall, precision, marker='.')
# show the plot
pyplot.show()

In [ ]:
'''
Created on May 6, 2018
@author: jana
'''
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from data_processing import load_data, prepareMolData, prepareMinibatches, context_dictionary, loadProteinRestrictions
from model import DeepVS
from scorer_auc_enrichment_factor import Scorer
import numpy as np
import random
import sys
import os


class DeepVSExperiment:
    
    def __init__(self,
        embedding_size = 200,
        cf = 400,
        h  = 50,
        lr = 0.00001,
        kc = 6,
        kp = 2,
        num_epochs = 7,
        minibatchSize = 20,
        l2_reg_rate = 0.0001,
        use_Adam = True):

        self.embedding_size = embedding_size
        self.cf = cf
        self.h = h 
        self.lr = lr
        self.kc = kc
        self.kp = kp
        self.num_epochs = num_epochs 
        self.minibatchSize = minibatchSize
        self.l2_reg_rate = l2_reg_rate
        self.use_Adam = use_Adam
        
        self._aucSumByEpoch       = [0.0]*10
        self._EfMaxSumByEpoch     = [0.0]*10
        self._Ef2SumByEpoch       = [0.0]*10
        self._Ef20SumByEpoch      = [0.0]*10
        self._numProteinProcessed = 0
        


    def run(self, datasetPath, proteinsNames_training, proteinsNames_test, proteinRestrictions):
        torch.manual_seed(31)   
        rng = random.Random(31)
        self.epoch = 0
        self._numProteinProcessed += 1.0


        testProRestrictions = proteinRestrictions.get(proteinsNames_test[0])
        
        if testProRestrictions is not None:
            i = len(proteinsNames_training) - 1
            while i > -1 :
                if proteinsNames_training[i] in testProRestrictions:
                    del proteinsNames_training[i]
                i -= 1
            
        # Preparing training dataset
        print("Loading data ...")
        molName_training, molClass_training, molData_training = load_data(datasetPath, self.kc, self.kp,
                                                                          proteinsNames_training, rng, randomize = True)
        #print(molName_training)
        #print(molData_training)
        print("proteinsNames_training: ", proteinsNames_training)
        print("Preparing data ...")
        context_to_ix_training = context_dictionary(molData_training)
        molData_ix_training = prepareMolData(molData_training, context_to_ix_training)
        molDataBatches_training = prepareMinibatches(molData_ix_training, molClass_training, self.minibatchSize)
        
#         print("CLASS", molClass_training[:20])
#         print("MOLD DATA TRAINING ", molData_training[:20])
#         print("CONTEXT_TO_IX_TRAINING FOR MOLDATA ", context_to_ix_training)
#         print("MOL DATA BATCHES FROM CONTEXT AND CLASS ", molDataBatches_training[:20])
        
        # Preparing test dataset
        molName_test, molClass_test, molData_test = load_data(datasetPath, self.kc, self.kp, proteinsNames_test,
                                                              rng, randomize = False)
        print("proteinsNames_test: ", proteinsNames_test)
        print("number of test molecules: ", len(molData_test))
        molData_ix_test = prepareMolData(molData_test, context_to_ix_training)
        molDataBatches_test = prepareMinibatches(molData_ix_test, molClass_test, self.minibatchSize)
        #print(proteinsNames_test)
        #molDataBatches_test = label_binarize(molDataBatches_test, classes=[0, 1])
        #print(molDataBatches_test)
        
        
        # Number of columns in the embedding matrix
        vocab_size = len(context_to_ix_training) 
        
        
        # Instantiate Model  Class
        model = DeepVS(vocab_size, self.embedding_size, self.cf, self.h, self.kc, self.kp)
        #print("---------VOCAB SIZE----------", vocab_size)
        #print("---------EMBEDDING size----------", self.embedding_size)
        #####################
        # Use GPU for model #
        #####################
        if torch.cuda.is_available():
            model.cuda()  
            print("using GPU!")
        
        
        # Instantiate Loss Class
        loss_fuction = nn.NLLLoss()
        
        
        # Instantiate scorer
        scorer = Scorer()
        
        # AUC SCORER
        # aucscorer = AUCScorer()
        
        
        # Instantiate optimizer class: using Adam
        if self.use_Adam:
            optimizer  = optim.Adam(model.parameters(), self.lr, weight_decay = self.l2_reg_rate)
            print('using Adam')            
        else:
            optimizer  = optim.SGD(model.parameters(), self.lr, weight_decay = self.l2_reg_rate)
            print('using SGD')
        
        print('lr = ', self.lr)
        print('ls_reg_rate = ', self.l2_reg_rate)
        
        
        

        # Train Model
        print("Training ...")
        for epoch in range(1, self.num_epochs+1):
            total_loss = 0.0
            model.train()
            for cmplx, cls, msk in molDataBatches_training:
                # convert contexts and classes into torch variables 
                if torch.cuda.is_available():
                    cls  = autograd.Variable(torch.LongTensor(cls).cuda())
                    cmplx = autograd.Variable(torch.LongTensor(cmplx).cuda())
                    mskv = autograd.Variable(torch.FloatTensor(msk).cuda())                  
                else:
                    cls  = autograd.Variable(torch.LongTensor(cls))
                    cmplx = autograd.Variable(torch.LongTensor(cmplx))
                    mskv = autograd.Variable(torch.FloatTensor(msk))
        
                model.zero_grad()
                                
                # Run the forwad pass 
                log_probs = model(cmplx, mskv)
                
                # Compute loss and update model 
                loss = loss_fuction(log_probs,cls)
                loss.backward()
                optimizer.step()
                total_loss += loss.data.cpu()        
            
            # shuffles the training set after each epoch
            rng.shuffle(molDataBatches_training)
            #rng.shuffle(molDataBatches_test)
            
            # sets model to eval (needed to use dropout in eval mode)
            model.eval()
            # Test model after each epoch
            correct = 0.0
            numberOfMolecules = 0.0
            total_loss_test = 0.0
            scores = []
            scr = []
            cls = []
            testMolId = 0
           # print(molDataBatches_test)
            for cmplx_test, cls_test, msk_test in molDataBatches_test:
               
                cls_test  = torch.LongTensor(cls_test)
                #print(cls_test)
                #print(cls_test)
                # convert contexts and classes into torch variables
                if torch.cuda.is_available():     
                    cls_test_v = autograd.Variable(cls_test.cuda())
                    cmplx_test = autograd.Variable(torch.LongTensor(cmplx_test).cuda())
                    mskv_test = autograd.Variable(torch.FloatTensor(msk_test).cuda())             
                else:
                    cls_test_v = autograd.Variable(cls_test)
                    cmplx_test = autograd.Variable(torch.LongTensor(cmplx_test))
                    mskv_test = autograd.Variable(torch.FloatTensor(msk_test)) 
                
                # Run the forwad pass 
                outputs = model(cmplx_test, mskv_test)
                loss_test = loss_fuction(outputs, cls_test_v)
                #print(outputs)
                #print(loss_test)
                

                #print(auc)
                # Get predictions 
                #print(outputs.data)
                #predict_mine = np.where(outputs.data > torch.LongTensor(0.3), torch.LongTensor(1), torch.LongTensor(0))
                _, predicted = torch.max(outputs.data, 1)
                #print("_, predicted ", _, predicted)

                for cur_scr, cur_cls in zip(np.exp(outputs.data[:,1]), cls_test.cpu()):
                    scr.append(cur_scr)
                    cls.append(cur_cls)
                    scores.append([cur_scr, cur_cls, molName_test[testMolId]])
                    #print(cur_scr)
                    testMolId += 1
                numberOfMolecules += cls_test.size()[0]
                
                
               
                
                      
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == cls_test.cpu()).sum()
                else:
                    correct += (predicted == cls_test).sum()
                    print("If predicted: ", predicted)
                    print("equals to test class: ",cls_test)
                    print("Correct = ", correct)
                    print("----------")      
                total_loss_test += loss_test.data
                
#             print("Total correct: ", correct)
            accuracy = 100 * correct / numberOfMolecules
            print("--------------------------------------------------------------------------------------------")    
            print("epoch = %d;  total loss training = %.4f; total loss test = %.4f; accuracy = %f" %(epoch, total_loss/len(molDataBatches_training), total_loss_test/len(molDataBatches_test), accuracy))
            efAll, dataForROCCurve, efValues, aucValue = scorer.computeEnrichmentFactor_and_AUC(scores,removeRepetitions=True)
#             aucScorer = AUCScorer(scoresToAuc)
            # -----
            # AUC
            # -----
            pos_0_scr= [x[0] for x in scores]
            pos_1_cls= [x[1] for x in scores]

            precision, recall, thresholds = precision_recall_curve(pos_1_cls, pos_0_scr) # cls, scr

            auc_score = auc(recall, precision)
            print("New AUC score: ", auc_score)
            
            # ---------
            # ROC curve
            # ---------
#----
#             scorer.plotROCCurve(dataForROCCurve, pos_1_cls)
#----
#----        
#             plt.plot([0, 1], [0.5, 0.5], linestyle='--')
#             # plot the precision-recall curve for the model
#             plt.plot(recall, precision, marker='.')
#             # show the plot
#             plt.show()
#---            
#----            
#             fpr, tpr, thresholds = roc_curve(pos_1_cls, pos_0_scr)
#             roc_auc = auc(fpr, tpr)
#             plt.figure()
#             plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
#             plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
#             plt.xlim([0.0, 1.0])
#             plt.ylim([0.0, 1.05])
#             plt.xlabel('False Positive Rate')
#             plt.ylabel('True Positive Rate')
#             plt.title('Receiver operating characteristic')
#             plt.legend(loc="lower right")
#             plt.show()
#----

            self._aucSumByEpoch[self.epoch]    += auc_score #aucValue
            self._EfMaxSumByEpoch[self.epoch]  += efValues[2]
            self._Ef2SumByEpoch[self.epoch]    += efValues[0]
            self._Ef20SumByEpoch[self.epoch]   += efValues[1]
            
            self.epoch += 1
            
            # -------
            # Confusion matrix
            # -------
#             print(accuracy_score(pos_1_cls, pos_0_scr))
#             print(confusion_matrix(pos_1_cls, pos_0_scr))
#             print(classification_report(pos_1_cls, pos_0_scr))
#             print("--------------Confusion matrix from DeepVS threshold = 0.5--------------")
            #aucScorer.confusion_matrix(threshold=0.5,do_print=True)
            
            
            
        print("Average AUC, EF2, EF20, EFMax by epoch for %d proteins:"%self._numProteinProcessed)
        for k in range(self.num_epochs):
            print("Ep: %d, AUC: %.4f -"%(k+1, self._aucSumByEpoch[k]/self._numProteinProcessed), end=' ')
        print(" ")
        for k in range(self.num_epochs):
            print("Ep: %d, EF 2%%: %.4f -"%(k+1, self._Ef2SumByEpoch[k]/self._numProteinProcessed), end=' ')
        print(" ")
        for k in range(self.num_epochs):
            print("Ep: %d, EF 20%%: %.4f -"%(k+1, self._Ef20SumByEpoch[k]/self._numProteinProcessed), end=' ')
        print(" ")
        for k in range(self.num_epochs):
            print("Ep: %d, EF Max: %.4f -"%(k+1, self._EfMaxSumByEpoch[k]/self._numProteinProcessed), end=' ')
        print(" ")
        sys.stdout.flush()


if __name__ == '__main__':

    '''
    Definition of Hyperparameters:
    
    embedding_size = embedding size of d^atm, d^amino, d^chg, d^dist
    cf = number of convolutional filters
    h =  number of hidden units
    lr = learning rate
    kc = number of neighboring atoms from compound
    kp = number of neighboring atoms from protein
    num_epoch = number of epochs
    '''

    
    dvsExp = DeepVSExperiment(
        embedding_size = 200,
        cf = 400,
        h  = 50,
        lr = 0.00001,
        kc = 6,
        kp = 2,
        num_epochs = 7,
        minibatchSize = 20,
        l2_reg_rate = 0.0001,
        use_Adam = True)
    
    
    
    
    proteinNames = ['ace', 
                    'ache', 'ada', 'alr2', 'ampc', 'ar']#, 'cdk2','comt', 'cox1']
#                     'cox2', 'dhfr', 'egfr', 'er_agonist', 'er_antagonist', 
#                     'fgfr1', 'fxa', 'gart', 'gpb', 'gr', 'hivpr', 'hivrt', 'hmga', 'hsp90', 'inha', 'mr', 
#                     'na', 'p38', 'parp', 'pde5', 'pdgfrb', 'pnp', 'ppar', 'pr', 'rxr', 'sahh', 
#                     'src', 'thrombin', 'tk', 'trypsin', 'vegfr2']
    
                    
#                     ['3klm', '3tfq', '1a4g', '1a5h', '1adw', '1ah3'
#                     ,'1b8o', '1ckp', '1cx2', '1e3g', '1eve', '1f0r', '1fm9', '1hw8', '1i00'
#                     ,'1j8f', '1m17', '1nhz', '1ouk', '1p44', '1r4l', '1s3v', '1s6p', '1uou'
#                     ,'1uy6', '1uze', '1w4r', '1xjd', '1xoi', '1xp0', '1z11', '2afx', '2b1p'
#                     ,'2dg3', '2iwi', '2oo8', '2p1t', '2p54', '2src', '2vgo', '2vwz', '2w31'
#                     ,'2w8y', '2wcg', '2xch', '2z94', '3bc3', '3c7q', '3dbs', '3dds', '3elj'
#                     ,'3eml', '3ewj', '3fdn', '3frg', '3hng', '3i4b', '3k5e', '3kc3', '3kk6'
#                     ,'3kx1', '3l3m', '3lbk', '3lxl', '3max', '3mhw', '3mj1', '3mpm', '3npc'
#                     ,'3nu3', '3nw7', '3ny9', '3oll', '3pp0', '3qkl', '3r04', '3rm2', '3sff'
#                     ,'3skc', '3v8s']
    
    
                    #['3tfq', '1a5h']
                    # ace
                    #'ache', 'ada', 'alr2', 'ampc', 'ar', 'cdk2','comt', 'cox1', 
                    #'cox2', 'dhfr', 'egfr', 'er_agonist', 'er_antagonist', 
                    #'fgfr1', 'fxa', 'gart', 'gpb', 'gr', 'hivpr', 'hivrt', 'hmga', 'hsp90', 'inha', 'mr', 
                    #'na', 'p38', 'parp', 'pde5', 'pdgfrb', 'pnp', 'ppar', 'pr', 'rxr', 'sahh', 
                    #'src', 'thrombin', 'tk', 'trypsin', 'vegfr2']
    
    
 
    datasetPath  = 'dud_vinaout_deepvs/'
    proteinGroupsFileName = 'protein.groups' 
    proteinCrossEnrichmentFileName = 'protein.cross_enrichment'
    
    
    proteinNames = [x for x in proteinNames if os.path.isfile(datasetPath+x+".deepvs")]
#     print(len(filtered))
#     for i, pr in enumerate(proteinNames):
# #         if(str(prot).__contains__("1hw8")):
# #             print("This: ",prot)
# #         else:
#         print(i, pr)
  
#         if(not os.path.isfile(datasetPath+pr.strip()+".deepvs")):
#             #print("File does not exist in directory: ", datasetPath+prot+".deepvs")
#            # print("Removing it from the list...")
#             proteinNames.remove(pr)
#     print(count)
    print("New protein Names: ", proteinNames)
    print("Length of protein list", len(proteinNames))

    proteinRestrictions = loadProteinRestrictions(proteinGroupsFileName,proteinCrossEnrichmentFileName)
    
    
    for pName in proteinNames:
        proteinNames_test = []
        proteinNames_training = ''
        if pName in proteinNames:
            proteinNames_test.append(pName)
            proteinNames_training = proteinNames[:]
            del proteinNames_training[proteinNames_training.index(pName)]
            
            print("======================================================================")
            print("Experimental results for protein:", proteinNames_test)
            print("======================================================================")
            dvsExp.run(datasetPath, proteinNames_training, proteinNames_test, proteinRestrictions)
